# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc6bc121ca0>
├── 'a' --> tensor([[ 1.4690,  1.1233,  0.8849],
│                   [ 0.0734,  1.1338, -0.5753]])
└── 'x' --> <FastTreeValue 0x7fc6bc1218b0>
    └── 'c' --> tensor([[ 0.7051,  1.3283,  0.6842, -1.3941],
                        [-0.1881,  0.0109, -0.2679,  0.6882],
                        [ 1.0708,  1.1079, -0.1145,  0.3876]])

In [4]:
t.a

tensor([[ 1.4690,  1.1233,  0.8849],
        [ 0.0734,  1.1338, -0.5753]])

In [5]:
%timeit t.a

77.5 ns ± 0.117 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc6bc121ca0>
├── 'a' --> tensor([[ 0.7072, -1.0218, -0.0289],
│                   [-0.3775,  1.1565, -1.0265]])
└── 'x' --> <FastTreeValue 0x7fc6bc1218b0>
    └── 'c' --> tensor([[ 0.7051,  1.3283,  0.6842, -1.3941],
                        [-0.1881,  0.0109, -0.2679,  0.6882],
                        [ 1.0708,  1.1079, -0.1145,  0.3876]])

In [7]:
%timeit t.a = new_value

85.7 ns ± 0.196 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.4690,  1.1233,  0.8849],
               [ 0.0734,  1.1338, -0.5753]]),
    x: Batch(
           c: tensor([[ 0.7051,  1.3283,  0.6842, -1.3941],
                      [-0.1881,  0.0109, -0.2679,  0.6882],
                      [ 1.0708,  1.1079, -0.1145,  0.3876]]),
       ),
)

In [10]:
b.a

tensor([[ 1.4690,  1.1233,  0.8849],
        [ 0.0734,  1.1338, -0.5753]])

In [11]:
%timeit b.a

73.2 ns ± 2.38 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3500,  0.4705, -2.0962],
               [ 1.3381, -0.9827,  1.4061]]),
    x: Batch(
           c: tensor([[ 0.7051,  1.3283,  0.6842, -1.3941],
                      [-0.1881,  0.0109, -0.2679,  0.6882],
                      [ 1.0708,  1.1079, -0.1145,  0.3876]]),
       ),
)

In [13]:
%timeit b.a = new_value

590 ns ± 0.153 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

991 ns ± 3.05 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.6 µs ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

166 µs ± 741 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

163 µs ± 472 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc617e40e80>
├── 'a' --> tensor([[[ 1.4690,  1.1233,  0.8849],
│                    [ 0.0734,  1.1338, -0.5753]],
│           
│                   [[ 1.4690,  1.1233,  0.8849],
│                    [ 0.0734,  1.1338, -0.5753]],
│           
│                   [[ 1.4690,  1.1233,  0.8849],
│                    [ 0.0734,  1.1338, -0.5753]],
│           
│                   [[ 1.4690,  1.1233,  0.8849],
│                    [ 0.0734,  1.1338, -0.5753]],
│           
│                   [[ 1.4690,  1.1233,  0.8849],
│                    [ 0.0734,  1.1338, -0.5753]],
│           
│                   [[ 1.4690,  1.1233,  0.8849],
│                    [ 0.0734,  1.1338, -0.5753]],
│           
│                   [[ 1.4690,  1.1233,  0.8849],
│                    [ 0.0734,  1.1338, -0.5753]],
│           
│                   [[ 1.4690,  1.1233,  0.8849],
│                    [ 0.0734,  1.1338, -0.5753]]])
└── 'x' --> <FastTreeValue 0x7fc6bc1137f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

36.9 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc60d017820>
├── 'a' --> tensor([[ 1.4690,  1.1233,  0.8849],
│                   [ 0.0734,  1.1338, -0.5753],
│                   [ 1.4690,  1.1233,  0.8849],
│                   [ 0.0734,  1.1338, -0.5753],
│                   [ 1.4690,  1.1233,  0.8849],
│                   [ 0.0734,  1.1338, -0.5753],
│                   [ 1.4690,  1.1233,  0.8849],
│                   [ 0.0734,  1.1338, -0.5753],
│                   [ 1.4690,  1.1233,  0.8849],
│                   [ 0.0734,  1.1338, -0.5753],
│                   [ 1.4690,  1.1233,  0.8849],
│                   [ 0.0734,  1.1338, -0.5753],
│                   [ 1.4690,  1.1233,  0.8849],
│                   [ 0.0734,  1.1338, -0.5753],
│                   [ 1.4690,  1.1233,  0.8849],
│                   [ 0.0734,  1.1338, -0.5753]])
└── 'x' --> <FastTreeValue 0x7fc6063b9f40>
    └── 'c' --> tensor([[ 0.7051,  1.3283,  0.6842, -1.3941],
                        [-0.1881,  0.0109, -0.2679,  0.6882],
                 

In [23]:
%timeit t_cat(trees)

35.2 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.4 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.4690,  1.1233,  0.8849],
                [ 0.0734,  1.1338, -0.5753]],
       
               [[ 1.4690,  1.1233,  0.8849],
                [ 0.0734,  1.1338, -0.5753]],
       
               [[ 1.4690,  1.1233,  0.8849],
                [ 0.0734,  1.1338, -0.5753]],
       
               [[ 1.4690,  1.1233,  0.8849],
                [ 0.0734,  1.1338, -0.5753]],
       
               [[ 1.4690,  1.1233,  0.8849],
                [ 0.0734,  1.1338, -0.5753]],
       
               [[ 1.4690,  1.1233,  0.8849],
                [ 0.0734,  1.1338, -0.5753]],
       
               [[ 1.4690,  1.1233,  0.8849],
                [ 0.0734,  1.1338, -0.5753]],
       
               [[ 1.4690,  1.1233,  0.8849],
                [ 0.0734,  1.1338, -0.5753]]]),
    x: Batch(
           c: tensor([[[ 0.7051,  1.3283,  0.6842, -1.3941],
                       [-0.1881,  0.0109, -0.2679,  0.6882],
                       [ 1.0708,  1.1079, -0.1145,  0.3876]],
         

In [26]:
%timeit Batch.stack(batches)

89.4 µs ± 507 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.4690,  1.1233,  0.8849],
               [ 0.0734,  1.1338, -0.5753],
               [ 1.4690,  1.1233,  0.8849],
               [ 0.0734,  1.1338, -0.5753],
               [ 1.4690,  1.1233,  0.8849],
               [ 0.0734,  1.1338, -0.5753],
               [ 1.4690,  1.1233,  0.8849],
               [ 0.0734,  1.1338, -0.5753],
               [ 1.4690,  1.1233,  0.8849],
               [ 0.0734,  1.1338, -0.5753],
               [ 1.4690,  1.1233,  0.8849],
               [ 0.0734,  1.1338, -0.5753],
               [ 1.4690,  1.1233,  0.8849],
               [ 0.0734,  1.1338, -0.5753],
               [ 1.4690,  1.1233,  0.8849],
               [ 0.0734,  1.1338, -0.5753]]),
    x: Batch(
           c: tensor([[ 0.7051,  1.3283,  0.6842, -1.3941],
                      [-0.1881,  0.0109, -0.2679,  0.6882],
                      [ 1.0708,  1.1079, -0.1145,  0.3876],
                      [ 0.7051,  1.3283,  0.6842, -1.3941],
                      [-0.1881,  

In [28]:
%timeit Batch.cat(batches)

166 µs ± 515 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

399 µs ± 18.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
